# Homework 1

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from datetime import timedelta 

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
pd.__version__

'2.2.1'

## Q1. Downloading the data

In [3]:
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_val = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
df_val.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

The dataset has 19 columns.

## Q2. Computing duration

In [7]:
df['duration'] = (df['tpep_dropoff_datetime']-df['tpep_pickup_datetime']).apply(lambda x: x.total_seconds()/60)

In [8]:
df['duration'].std()

42.59435124195458

The standard deviation of the duration is ~42.59 minutes.

## Q3. Dropping outliers

In [9]:
((df.duration >=1) & (df.duration <=60)).mean()

0.9812202822125979

We can already tell that the percentage of records remaining after outliers are removed is about 98%.

In [10]:
df = df[(df.duration >=1) & (df.duration <=60)]

## Q4. One-hot encoding

In [11]:
id_variables = ['PULocationID','DOLocationID']
vectorizer = DictVectorizer()

In [12]:
df[id_variables] = df[id_variables].astype('str')
dict_train = df[id_variables].to_dict(orient='records')

In [13]:
X_train = vectorizer.fit_transform(dict_train)

In [14]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

The matrix has 515 columns.

## Q5. Training a model

In [15]:
lr = LinearRegression()
y_train = df['duration']

In [16]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)

In [17]:
print('The RMSE of the model on the training data is', np.sqrt(mean_squared_error(y_train,y_pred)))

The RMSE of the model on the training data is 7.649261934850555


## Q6. Evaluating the model

In [18]:
df_val['duration'] = (df_val['tpep_dropoff_datetime']-df_val['tpep_pickup_datetime']).apply(lambda x: x.total_seconds()/60)
df_val = df_val[(df_val.duration >=1) & (df_val.duration <=60)]
y_val = df_val['duration']

In [19]:
df_val[id_variables] = df_val[id_variables].astype('str')
dict_val = df_val[id_variables].to_dict(orient='records')

In [20]:
X_val = vectorizer.transform(dict_val)
y_pred_val = lr.predict(X_val)

In [21]:
print('The RMSE of the model on the validation data is', np.sqrt(mean_squared_error(y_val,y_pred_val)))

The RMSE of the model on the validation data is 7.811817745843695
